In [1]:
import os
import pandas as pd
import numpy as np

from timeit import default_timer as timer
from datetime import timedelta

import nibabel as nib

from sklearn.model_selection import train_test_split
from imblearn import over_sampling 

from itertools import combinations
from multiprocessing import Pool, cpu_count

from src.data_balancing import data_balancing
import warnings

warnings.filterwarnings('ignore')
   

In [2]:
ROOT_DATA =r'C:\Users\anaga\OneDrive\Universidade\5º ano\Tese'
SES='ses-01'
TASK = 'task-01'
TR = 1

#### Set variables

In [24]:
SUB = 'sub-04'

labels_type = 'ptcps' #'predef' (predefined labels) or 'ptcps' (participant's labels)

file_name_format = SUB + '_' + SES + '_' + TASK

In [25]:
if labels_type == 'ptcps': 

    # data folder
    results_path = os.path.join(ROOT_DATA, 'results','voxel_stability','participant_labels')

elif labels_type == 'predef':
    results_path = os.path.join(ROOT_DATA, 'results','voxel_stability','predefined_labels')




### Load data

In [26]:
# Image with all runs of a participant not balanced
imgs_not_balanced_fn = os.path.join(results_path,SUB, file_name_format +'_allruns_VS.nii.gz')

# 4D image
imgs_not_balanced = nib.load(imgs_not_balanced_fn)

# Image signal data
data_not_balanced = imgs_not_balanced.get_fdata()

# Number of voxels
n_voxels = np.prod(data_not_balanced.shape[:3])

# Number of volumes
n_vols = data_not_balanced.shape[3]

#Convert 4D image to 2D image
data_notbalanced_2d = np.reshape(data_not_balanced, (n_voxels, n_vols))
data_notbalanced_2d = np.transpose(data_notbalanced_2d)


# Events dataframe with all runs of a participant not balanced
events_not_balanced_fn = os.path.join(results_path,SUB,file_name_format + '_events_all_runs_VS.csv')

# Read events dataframe
events_not_balanced=pd.read_csv(events_not_balanced_fn,sep=';',index_col = 0)

print('Length of events_not_balanced df: ',len(events_not_balanced))
print('Image 2D not balanced shape: 2d  ',data_notbalanced_2d.shape)
print('Image 4D not balanced shape: 4d  ',imgs_not_balanced.shape)



Length of events_not_balanced df:  128
Image 2D not balanced shape: 2d   (128, 1082035)
Image 4D not balanced shape: 4d   (97, 115, 97, 128)


### Create training and test sets

In [27]:
img_train, img_test, events_train, events_test = train_test_split(data_notbalanced_2d, events_not_balanced, stratify = events_not_balanced['trial_type'],test_size=0.30)

In [28]:
events_train['trial_type'].value_counts()

Noise    22
Q4       21
Q2       19
Q1       19
Q3        8
Name: trial_type, dtype: int64

 ### Oversampling the training data or not

In [29]:
if labels_type == 'ptcps':

    # get the name and the counts of the label with minimum counts in the training set
    min_label = events_train['trial_type'].value_counts().idxmin()
    min_count = events_train['trial_type'].value_counts().min()

    # oversample the label with minimum counts in the training set
    oversample = over_sampling.SMOTE(sampling_strategy = {min_label: 12}, k_neighbors=min_count-1)

    if min_count>=12:
        X_resampled, y_resampled = img_train,events_train['trial_type'].values
        
    else:
        # fit resample 
        X_resampled, y_resampled = oversample.fit_resample(img_train,events_train['trial_type'])


        # Turn y_resampled into a dataframe
        y_resampled_df = pd.DataFrame(y_resampled, columns = ['trial_type'])

        # If there is more than one label with less than 12 counts:

        # get the name and the counts of the label with minimum counts in the training set
        second_min_label = y_resampled_df['trial_type'].value_counts().idxmin()
        second_min_count = y_resampled_df['trial_type'].value_counts().min()

        if second_min_count < 12:
            # oversample the label with minimum counts in the training set
            oversample = over_sampling.SMOTE(sampling_strategy = {second_min_label: 12}, k_neighbors=second_min_count-1)

            # fit resample 
            X_resampled, y_resampled = oversample.fit_resample(X_resampled,y_resampled_df['trial_type'])


    # Turn y_resampled into a dataframe
    y_resampled_df = pd.DataFrame(y_resampled, columns = ['trial_type'])

    # Convert the 2d image back to 4d
    n_vols = X_resampled.shape[0]
    shape = list(data_not_balanced.shape[0:3])

    #Add the number of volumes to the shape
    shape.append(n_vols)

    # Convert X_resampled to 4d
    X_resampled_4d = np.reshape(X_resampled, shape)

    resampled_nii_img = nib.Nifti1Image(X_resampled_4d, imgs_not_balanced.affine, imgs_not_balanced.header)


    img_4d_fn = SUB + '_' + SES + '_' + TASK +'_allruns_train_VS.nii.gz'
    nib.save(resampled_nii_img, f'{results_path}/{img_4d_fn}')

    X_train = X_resampled
    y_train = y_resampled_df

else:
    y_train = events_train['trial_type']
    X_train = img_train

### Balance training data

In [34]:
# allruns='no' para não fazer o balanceamento dentro de cada run
img_train_balanced,events_train_balanced = data_balancing(X_train,y_train,'no','no')


print('img_train_balanced shape', img_train_balanced.shape)
print('events_train_balanced length: ',len(events_train_balanced))

events_train_balanced['trial_type'].value_counts()

img_train_balanced shape (60, 1082035)
events_train_balanced length:  60


Q2       12
Noise    12
Q1       12
Q4       12
Q3       12
Name: trial_type, dtype: int64

#### Save train and test sets


In [ ]:
events_train_fn = os.path.join(results_path,file_name_format+'events_train.csv')
events_train_balanced.to_csv(events_train_fn,sep=';')

events_test_fn = os.path.join(results_path,file_name_format+'events_test.csv')
events_test.to_csv(events_test_fn,sep=';')

img_train_fn = os.path.join(results_path,file_name_format+'img_data_train.npy')
np.save(img_train_fn,img_train_balanced)

img_test_fn = os.path.join(results_path,file_name_format+'img_data_test.npy')
np.save(img_test_fn,img_test)


### Organize the signal values of each voxel in each Noise, Q1,Q2,Q3,Q4 block


In [35]:
reorg_df = events_train_balanced.copy()

# Group by trial_type and create a new column with the order of each value within its group
reorg_df['order'] = reorg_df.groupby('trial_type').cumcount()

# Sort by the order column to reorder the rows within each group
reorg_df = reorg_df.sort_values(['order', 'trial_type'])

reorg_df.drop('order', axis=1, inplace=True)

#Indexes da tabela original na ordem que corresponde à ordenação dos trial_types (para ordenar o sinal dos voxeis)
reorg_indexes = reorg_df.index

#Reordenar o array 2d com o sinal de todos os voxeis para corresponder à nova ordem da data frame
reorg_2d_data = np.transpose(img_train_balanced[reorg_indexes,:])

# Colunas da nova dataframe são os trial_types reorganizados 
trial_types = reorg_df['trial_type']
# Criar uma dataframe com o sinal de todos os voxeis
voxels_df = pd.DataFrame(data=reorg_2d_data, columns=trial_types)
voxels_df



trial_type,Noise,Q1,Q2,Q3,Q4,Noise,Q1,Q2,Q3,Q4,...,Noise,Q1,Q2,Q3,Q4,Noise,Q1,Q2,Q3,Q4
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1082031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1082032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1082033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Calculates stability of each voxel

In [ ]:
# init timer()
fit_start=timer()

#Creates a new sub-dataframe for each voxel in which each row corresponds to a block of stimuli

set_size = 5 #Set size ['Noise', 'Q1','Q2','Q3','Q4']

#Number of sets of 5 blocks
sets_number = int((voxels_df.shape[1] / set_size))
sets_number

voxels_stability = []

for voxel in range(len(voxels_df)):
#for voxel in range(0,1):

    voxel_sets = []

    # Separates the dataframe into dataframes corresponding to each trial (presentation set) to reorganize the data
    for i in range(0,voxels_df.shape[1] , set_size):
        df_trial = voxels_df.iloc[voxel, i:i+set_size]
        voxel_sets.append(df_trial)


    voxel_all_sets = pd.concat(voxel_sets)
    voxel_all_sets_df  = pd.DataFrame(np.array(voxel_all_sets).reshape(sets_number,5))

    # Assign column names
    voxel_all_sets_df.columns = ['Noise', 'Q1', 'Q2', 'Q3', 'Q4']
    #display(voxel_all_sets_df)


    # Determines all possible combinations between each stimulus block (there are 8 blocks -> 8 choose 2 = 28 combinations)
    combs = list(combinations(voxel_all_sets_df.index, 2))
    #print('len combs: ',len(combs))

    #Pairwise correlations between the responses of each voxel
    voxel_corr = []
    for i, j in combs:
        #Pairwise correlations between the presentation sets
        corr = np.corrcoef(voxel_all_sets_df.iloc[i], voxel_all_sets_df.iloc[j])[0,1]
        voxel_corr.append(corr)

    #The stability of a voxel is the average of the pairwise correlations between the possible combinations of blocks
    voxel_stability = np.mean(voxel_corr)
    voxels_stability.append(voxel_stability)

    #Print stability from 10000 to 10000 voxels
    if (voxel) % 10000 == 0:
        print('Voxel {} stability: {:.6f} '.format(voxel+1,voxel_stability))
    #print('Voxel {} stability: {:.6f} '.format(voxel+1,voxel_stability))

# stop timer()
fit_end=timer()

# print elapsed time for model training
print(timedelta(seconds=fit_end-fit_start))

####  Saves dataframe with stability of all voxels

In [ ]:
# Dataframe with the stabolity of each voxel
all_voxels_stability = pd.DataFrame({'voxel': range(len(voxels_df)),'stability': voxels_stability})

#Save dataframe with all voxels stability
all_voxels_fn = os.path.join(results_path,SUB,file_name_format+'_stability_all_voxels.csv')
all_voxels_stability.to_csv(all_voxels_fn,sep=';')

#### Saves dataframe with the stability of the n most stable voxels

In [ ]:
#Load of the dataframe with the stability of all voxels
all_voxels_stability_fn = os.path.join(results_path,SUB,file_name_format+'_stability_all_voxels.csv')
all_voxels_stability = pd.read_csv(all_voxels_stability_fn,sep=';',index_col=0)

In [ ]:
nvoxels_list = np.array(range(100,1100,100))
nvoxels_list = [100,200,300,400,500,600,700,800,900,1000,5000,10000,100000,900000]


#Selects the n most stable voxels
for n in nvoxels_list:
    top_voxels = all_voxels_stability.nlargest(n, 'stability')
    most_stable_voxels = top_voxels.reset_index(drop=True)

    #Saves the dataframe with the stability and the indexes of the most stable voxels
    most_stable_voxels_fn = os.path.join(results_path,SUB,file_name_format+'_'+str(n)+'_most_stable_voxels.csv')
    most_stable_voxels.to_csv(most_stable_voxels_fn,sep=';')

### Creating mask with most n stable voxels
#### Apply mask to test and training sets

In [ ]:
for n in nvoxels_list:
    
    print('n: ',n)
    
    #Load and setting the indexes of the most stable voxels to a numpy array
    most_stable_voxels_fn = os.path.join(results_path,SUB,file_name_format+'_'+str(n)+'_most_stable_voxels.csv')
    most_stable_voxels = np.array(pd.read_csv(most_stable_voxels_fn,sep=';',index_col=0)['voxel'])
    print('stable_voxels.shape: ',most_stable_voxels.shape)

    #Load of the training set
    img_train_fn = os.path.join(results_path,SUB,file_name_format+'img_data_train.npy')
    img_train = np.load(img_train_fn)
    print('img_train shape:',img_train.shape)

    #Load of the testing set
    img_test_fn = os.path.join(results_path,SUB,file_name_format+'img_data_test.npy')
    img_test = np.load(img_test_fn)
    print('img_test shape:',img_test.shape)

    #Masked test set
    masked_test_set = img_test[:,most_stable_voxels]
    print('masked_test_set shape:',masked_test_set.shape)

    #Masked training set
    masked_training_set = img_train[:,most_stable_voxels]
    print('masked_training_set shape:',masked_training_set.shape)
    
    #Save masked training set only with stable voxels
    masked_train_fn = os.path.join(results_path,SUB,file_name_format+'_'+str(n)+'voxels_train_set.npy')
    np.save(masked_train_fn, masked_training_set)
    
    #Save test testing set only with stable voxels
    masked_test_fn = os.path.join(results_path,SUB,file_name_format+'_'+str(n)+'voxels_test_set.npy')
    np.save(masked_test_fn, masked_test_set)
    
    #Load of the 4D image with all voxels
    img_all_voxels = nib.load(imgs_not_balanced_fn)
    print('imgs_all.shape: ',img_all_voxels.shape)
    
    
    # Inicialize mask with 0's
    mask_2d = np.zeros(data_notbalanced_2d.shape[1])

    #The indexes that correspond to the stable voxels are set to 1
    mask_2d[most_stable_voxels]=1
    print('mask_2d.shape: ',mask_2d.shape)

    # Image reshape from 2D to 3D
    mask_3d = mask_2d.reshape(img_all_voxels.shape[:3])
    print('mask_3d.shape: ',mask_3d.shape)

    #New nifti image with the same header information as the original image
    mask_img = nib.Nifti1Image(mask_3d, img_all_voxels.affine, img_all_voxels.header)
    print('mask_img.shape: ',mask_img.shape)

    #Save mask image with n voxels
    mask_img_fn = file_name_format+'_'+str(n)+'voxels_img3D_VS.nii.gz'
    nib.save(mask_img, f'{results_path}/{mask_img_fn}')
